Author: Lindsay Spratt

# Function to calculate weighted Elixhauser comorbidity for an individual patient within a specified time window

## References:
Davis, M. (n.d.). Mcdgit29/pyelixhauser: Python package to detect clinical morbodity in ICD10 and ICD9 codes. Retrieved from https://github.com/mcdgit29/pyelixhauser

Elixhauser, A., Steiner, C., Harris, D. R., & Coffey, R. M. (1998). Comorbidity Measures for Use with Administrative Data. Medical Care, 36(1), 8–27. http://www.jstor.org/stable/3766985

Elixhauser Comorbidity Software Refined for ICD-10-CM. Accessed May 20, 2024. https://hcup-us.ahrq.gov/toolssoftware/comorbidityicd10/comorbidity_icd10.jsp

In [ ]:
# Weights of conditions obtained from https://hcup-us.ahrq.gov/toolssoftware/comorbidityicd10/CMR-User-Guide-v2022-1.pdf
# *** These weights reflect the in-hospital mortality index ***

# HTN_UNCX was removed from HCUP's model because it had a "very low observed in-hospital mortality rate and were highly correlated with another
# comorbidity measure". Becasuse of this, I have set HTN_UNCX weight to 0 since it is included in the pyelixhauser package

elixhauser_weights = {
    'AIDS': -4,
    'ALCOHOL': -1,
    'ANEMDEF': -3,
    'AUTOIMMUNE': -1,
    'BLDLOSS': -4,
    'CANCER_LEUK': 9,
    'CANCER_LYMPH': 6,
    'CANCER_METS': 23,
    'CANCER_NSITU': 0,
    'CANCER_SOLID': 10,
    'CBVD_POA': 5,
    'CBVD_SQLA': 5,
    'COAG': 15,
    'DEMENTIA': 5,
    'DEPRESS': -9,
    'DIAB_CX': -2,
    'DIAB_UNCX': 0,
    'DRUG_ABUSE': -7,
    'HF': 15,
    'HTN_CX': 1,
    'HTN_UNCX': 0,
    'LIVER_MLD': 2,
    'LIVER_SEV': 17,
    'LUNG_CHRONIC': 2,
    'NEURO_MOVT': -1,
    'NEURO_OTH': 23,
    'NEURO_SEIZ': 2,
    'OBESE': -7,
    'PARALYSIS': 4,
    'PERIVASC': 3,
    'PSYCHOSES': -9,
    'PULMCIRC': 4,
    'RENLFL_MOD': 3,
    'RENLFL_SEV': 8,
    'THYROID_HYPO': -3,
    'THYROID_OTH': -8,
    'ULCER_PEPTIC': 0,
    'VALVE': 0,
    'WGHTLOSS': 14
}
# From https://hcup-us.ahrq.gov/toolssoftware/comorbidityicd10/CMR-User-Guide-v2022-1.pdf
# "Some comorbidity weights carry a negative sign, reflecting a
# protective relationship with in-hospital mortality in the context of the model."

In [ ]:
# Function to calculate weighted Elixhauser comorbidity for an individual person within a specified time window
def calculate_weighted_elixhauser_for_time_windows(person_id, time_windows):
    # Filter data for the individual person
    condition_person_df = condition_df[condition_df['person_id'] == person_id]
    person_info = person_df[person_df['person_id'] == person_id].iloc[0]
    
    # Container to hold results
    results = []

    for start_date, end_date in time_windows:
        # Filter conditions within the time window
        filtered_conditions_df = condition_person_df[
            (condition_person_df['condition_start_datetime'] >= start_date) & 
            (condition_person_df['condition_start_datetime'] <= end_date)
        ]
        

        # Aggregate condition codes into a single string
        condition_codes = ' '.join(filtered_conditions_df['source_concept_code'].astype(str))

        # Calculate comorbidity index for the person within the time window
        comorbidity_results = comorbidity_from_string(condition_codes)

        # Convert the results to a DataFrame
        df = pd.DataFrame([comorbidity_results], index=[person_id])

        # Ensure the DataFrame columns are numeric
        df = df.apply(pd.to_numeric)

        # Add time window information
        df['start_date'] = start_date
        df['end_date'] = end_date

        # Calculate the weighted comorbidity index
        df['weighted_comorbidity_index'] = sum(df[col] * elixhauser_weights.get(col, 0) for col in df.columns if col in elixhauser_weights)

        # Append results to the list
        results.append(df)

    # Concatenate all results into a single DataFrame
    results_df = pd.concat(results)

    # Merge comorbidity results with person demographic data
    person_info_df = pd.DataFrame([person_info])
    person_info_with_comorbidity = pd.concat([person_info_df.set_index('person_id'), results_df], axis=1)

    return person_info_with_comorbidity


# Define time windows 
time_windows = [
    (pd.Timestamp('2008-01-01'), pd.Timestamp('2020-06-01')),
    (pd.Timestamp('2021-01-01'), pd.Timestamp('2021-06-01'))
]

# Calculate weighted Elixhauser comorbidity for a specific person over multiple time windows
person_id = 1052046
person_comorbidity_df = calculate_weighted_elixhauser_for_time_windows(person_id, time_windows)

# Display the results
print(person_comorbidity_df)